In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import xarray as xr

matplotlib.rcParams.update({'font.size': 14})
import dedalus.public as d3

import logging
logging.disable()

In [44]:
from frontal_zone import *

In [45]:
n_wavenumbers = 50; wavenumbers = 10. **np.linspace(-5, -2, n_wavenumbers)

## Sensitivity of growth rate $\Im\{\omega\}$ and instability scale $\lambda_{\text{max}}$ to frontal strength $M$

In [46]:
Mc = N*np.sqrt(np.sin(θ))
print(fr"We consider only subcritical fronts, i.e. with M < Mc = {round(Mc,7)}, which are gravitationally stable.")

ms = np.logspace(0, 5, 6)
Ms = calc_M_from_m(ms, Mc)
gr = xr.DataArray(np.zeros((n_wavenumbers, Ms.size)), coords={'wavenumber':wavenumbers, 'M':Ms,})
for M in Ms:
    for n in wavenumbers:
        bfzi = bottom_frontal_zone_instability(0., 2*np.pi*n, M, κ0=1.e-7, κ1=0.)
        gr = gr.where(~((gr.M==M) & (gr.wavenumber==n)), bfzi['omega'][bfzi['idx']].imag)
gr.to_netcdf("../../data/growth_rates/frontal_strength_M.nc")

We consider only subcritical fronts, i.e. with M < Mc = 4.47e-05, which are gravitationally stable.


## Sensitivity to frictional parameters ($\nu, \kappa, \nu_{h}, \kappa_{h}, \nu_{4}, \kappa_{4}$)

In [47]:
m = 100
M = calc_M_from_m(m, Mc)

gr = xr.Dataset()
gr.attrs["m"] = m
gr.attrs["M"] = M
friction_scalings = np.array([0., 1.e-2, 1e-1, 1.e0, 1e1, 1.e2, 1.e3])

reference_friction = {
    "κ0":κ0, "κ1":κ1, "νh":0.1, "ν4":2.e5,
}

##### Inviscid control

In [48]:
gr['inviscid'] = xr.DataArray(np.zeros((n_wavenumbers,)), coords={'wavenumber':wavenumbers,})
for n in wavenumbers:
    bfzi = bottom_frontal_zone_instability(0., 2*np.pi*n, M, κ0=1.e-7, κ1=0.)
    gr['inviscid'] = gr['inviscid'].where(~(gr.wavenumber==n), bfzi['omega'][bfzi['idx']].imag)

##### Only background slope-normal diffusivity and viscosity

In [ ]:
gr['k0'] = xr.DataArray(np.zeros((n_wavenumbers, friction_scalings.size)), coords={'wavenumber':wavenumbers, 'friction_scaling':friction_scalings,})
for sc in friction_scalings:
    for n in wavenumbers:
        bfzi = bottom_frontal_zone_instability(0., 2*np.pi*n, M, κ0=reference_friction["κ0"]*sc, κ1=0.)
        gr['k0'] = gr['k0'].where(~((gr.friction_scaling==sc) & (gr.wavenumber==n)), bfzi['omega'][bfzi['idx']].imag)

In [ ]:
gr['inviscid'].plot()
plt.xscale("log")
plt.yscale("log")
for sc in friction_scalings:
    gr['k0'].sel(friction_scaling=sc).plot(label=f"{sc}")
    
plt.legend()
plt.grid(True)

##### Variable bottom-enhancement of diffusivity and viscosity (on top of weak background)

In [ ]:
gr['k1'] = xr.DataArray(np.zeros((n_wavenumbers, friction_scalings.size)), coords={'wavenumber':wavenumbers, 'friction_scaling':friction_scalings,})
for sc in friction_scalings:
    for n in wavenumbers:
        bfzi = bottom_frontal_zone_instability(0., 2*np.pi*n, M, κ0=reference_friction["κ0"], κ1=reference_friction["κ1"]*sc)
        gr['k1'] = gr['k1'].where(~((gr.friction_scaling==sc) & (gr.wavenumber==n)), bfzi['omega'][bfzi['idx']].imag)

In [ ]:
gr['inviscid'].plot()
plt.xscale("log")
plt.yscale("log")
for sc in friction_scalings:
    gr['k1'].sel(friction_scaling=sc).plot(label=f"{sc}")
    
plt.legend()

##### Variable horizontal Laplacian viscosity

In [ ]:
gr['nuh'] = xr.DataArray(np.zeros((n_wavenumbers, friction_scalings.size)), coords={'wavenumber':wavenumbers, 'friction_scaling':friction_scalings,})
for sc in friction_scalings:
    for n in wavenumbers:
        bfzi = bottom_frontal_zone_instability(0., 2*np.pi*n, M, κ0=1.e-7, κ1=0., νh=reference_friction["νh"]*sc)
        gr['nuh'] = gr['nuh'].where(~((gr.friction_scaling==sc) & (gr.wavenumber==n)), bfzi['omega'][bfzi['idx']].imag)

In [ ]:
gr['inviscid'].plot()
plt.xscale("log")
plt.yscale("log")
for sc in friction_scalings:
    gr['nuh'].sel(friction_scaling=sc).plot(label=f"{sc}")
    
plt.legend()

##### Variable horizontal hyper-viscosity (w/ background diffusivities)

In [ ]:
gr['nu4'] = xr.DataArray(np.zeros((n_wavenumbers, friction_scalings.size)), coords={'wavenumber':wavenumbers, 'friction_scaling':friction_scalings,})
for sc in friction_scalings:
    for n in wavenumbers:
        bfzi = bottom_frontal_zone_instability(0., 2*np.pi*n, M, κ0=1.e-7, κ1=0., ν4=reference_friction["ν4"]*sc)
        gr['nu4'] = gr['nu4'].where(~((gr.friction_scaling==sc) & (gr.wavenumber==n)), bfzi['omega'][bfzi['idx']].imag)

In [ ]:
gr['inviscid'].plot()
plt.xscale("log")
plt.yscale("log")
for sc in friction_scalings:
    gr['nu4'].sel(friction_scaling=sc).plot(label=f"{sc}")
    
plt.legend()

##### Overly-viscous case

In [ ]:
gr['viscous'] = xr.DataArray(np.zeros((n_wavenumbers,)), coords={'wavenumber':wavenumbers,})
for n in wavenumbers:
    bfzi = bottom_frontal_zone_instability(
        0., 2*np.pi*n, M,
        νh=reference_friction["νh"], ν4=reference_friction["ν4"]
    )
    gr['viscous'] = gr['viscous'].where(~(gr.wavenumber==n), bfzi['omega'][bfzi['idx']].imag)

### Saving calculations for post-processing

In [ ]:
gr.to_netcdf("../../data/growth_rates/friction.nc")